In [1]:
import os
import re
import pickle
import pathlib
import pandas as pd
import arviz as az
import numpy as np
import palettes

from jax import numpy as jnp
from collections import namedtuple
from matplotlib import pyplot as plt

pd.set_option('display.max_rows', 500)
palettes.set_theme()

In [2]:
results_path = "./results"

In [3]:
def open_pickle(fl):
    with open(fl, "rb") as handle:
        d = pickle.load(handle)
    return d

In [4]:
dir = pathlib.Path(results_path)
results_files = []
for p in dir.rglob("*"):
    if str(p).endswith("results.pkl"):
        results_files.append(p)

In [5]:
model_reg = re.compile(".*/(.*)-n_dim_(\d+)-n_samples_(\d+)-(.*?)-.*")

dfs = pd.DataFrame()
for file in results_files:
    df = open_pickle(file)
    data = model_reg.match(str(file)).group(1)
    n_dim = int(model_reg.match(str(file)).group(2))
    n_samples = int(model_reg.match(str(file)).group(3))
    model = model_reg.match(str(file)).group(4)
    df.insert(0, "model", model)
    df.insert(0, "n_samples", n_samples)
    df.insert(0, "n_dim", n_dim)
    df.insert(0, "data_set", data)     
    dfs = pd.concat([dfs, df])

dfs.mse = dfs.mse.values.astype(np.float32)
dfs = dfs.fillna(0)
dfs = dfs.sort_values(["data_set", "n_dim", "n_samples", "model",]).drop(columns=["n_params"])

## Mean model

\begin{equation}
\begin{split}
\boldsymbol \mu &\sim \text{MvNormal}(\mathbf{0}, \mathbf{I}) \\
\mathbf{y}_n &\sim \text{MvNormal}(\boldsymbol \mu, \mathbf{I}) \qquad \forall n = 1, \dots, N
\end{split}
\end{equation}

In [6]:
mean_model = dfs[dfs.data_set == "mean_model"].drop(columns=["data_set"]).groupby(["n_dim", "n_samples", "model", "n_diffusions", "n_solver_steps", "n_solver_order"]).mean().round(2)
mean_model = mean_model.reset_index()
mean_model = mean_model.drop(columns=["n_dim"]).pivot_table(index=["model", "n_diffusions", "n_solver_steps", "n_solver_order"], columns=["n_samples"])
mean_model = mean_model.swaplevel(0, 1, axis=1).sort_index(axis=1)
mean_model = mean_model.reindex([
    (100, "elapsed_time_for_training"),
    (100, "elapsed_time_for_sampling"),
    (100, "mse"),
    (1000, "elapsed_time_for_training"),
    (1000, "elapsed_time_for_sampling"),
    (1000, "mse")
], axis=1)
mean_model

n_samples                                                             100   \
                                                 elapsed_time_for_training   
model n_diffusions n_solver_steps n_solver_order                             
advi  0.0          0.0            0.0                                 5.64   
ddpm  50.0         10.0           1.0                                42.16   
                                  3.0                                53.66   
                   20.0           1.0                                72.05   
                                  3.0                                75.19   
      100.0        10.0           1.0                                54.52   
                                  3.0                                60.93   
                   20.0           1.0                                92.94   
                                  3.0                                96.60   
flow  0.0          0.0            0.0                                35.34   

n_samples                                                                   \
                                                 elapsed_time_for_sampling   
model n_diffusions n_solver_steps n_solver_order                             
advi  0.0          0.0            0.0                                 0.28   
ddpm  50.0         10.0           1.0                                 4.00   
                                  3.0                                 7.35   
                   20.0           1.0                                 6.24   
                                  3.0                                10.39   
      100.0        10.0           1.0                                 5.88   
                                  3.0                                 9.43   
                   20.0           1.0                                 9.36   
                                  3.0                                14.23   
flow  0.0          0.0            0.0                                 1.13   

n_samples                                               \
                                                   mse   
model n_diffusions n_solver_steps n_solver_order         
advi  0.0          0.0            0.0             0.11   
ddpm  50.0         10.0           1.0             0.07   
                                  3.0             0.07   
                   20.0           1.0             0.07   
                                  3.0             0.07   
      100.0        10.0           1.0             0.07   
                                  3.0             0.07   
                   20.0           1.0             0.07   
                                  3.0             0.07   
flow  0.0          0.0            0.0             0.09   

n_samples                                                             1000  \
                                                 elapsed_time_for_training   
model n_diffusions n_solver_steps n_solver_order                             
advi  0.0          0.0            0.0                                19.59   
ddpm  50.0         10.0           1.0                               270.94   
                                  3.0                               256.45   
                   20.0           1.0                               565.43   
                                  3.0                               429.22   
      100.0        10.0           1.0                               353.74   
                                  3.0                               323.14   
                   20.0           1.0                               700.45   
                                  3.0                               607.68   
flow  0.0          0.0            0.0                               196.55   

n_samples                                                                   \
                                                 elapsed_time_for_sampling   
model n_diffusions n_solver_steps n_solver_o

## Mixture model

\begin{equation}
\begin{split}
\mu_{ki}        & \sim  \text{Normal}(0, 1) \qquad \forall k = 1, \dots, 3 \qquad \forall i = 1, \dots, 2  \\
\sigma_{ki}  & \sim  \text{HalfNormal}(1)  \qquad \forall k = 1, \dots, 3 \qquad \forall i = 1, \dots, 2  \\
y_{n}         & \sim \prod_{k=1}^K \pi_k \text{MvNormal}(\boldsymbol \mu_k, \boldsymbol \Sigma_k)\,\; \quad \forall n = 1, \dots, N
\end{split}
\end{equation}

In [7]:
mix_model = dfs[dfs.data_set == "mixture_model"].drop(columns=["data_set"]).groupby(["n_samples", "n_dim", "model", "n_diffusions", "n_solver_steps", "n_solver_order"]).mean().round(2)
mix_model = mix_model.reset_index()
mix_model = mix_model.pivot_table(index=["model", "n_diffusions", "n_solver_steps", "n_solver_order"], columns=["n_samples"])
mix_model = mix_model.swaplevel(0, 1, axis=1).sort_index(axis=1)
mix_model = mix_model.reindex([
    (100, "elapsed_time_for_training"),
    (100, "elapsed_time_for_sampling"),
    (100, "mse"),
    (1000, "elapsed_time_for_training"),
    (1000, "elapsed_time_for_sampling"),
    (1000, "mse")
], axis=1)
mix_model

n_samples                                                             100   \
                                                 elapsed_time_for_training   
model n_diffusions n_solver_steps n_solver_order                             
advi  0.0          0.0            0.0                                 6.77   
ddpm  50.0         10.0           1.0                                52.23   
                                  3.0                                56.92   
                   20.0           1.0                                77.19   
                                  3.0                                89.55   
      100.0        10.0           1.0                                56.17   
                                  3.0                                67.96   
                   20.0           1.0                                99.49   
                                  3.0                                96.35   
flow  0.0          0.0            0.0                                78.84   

n_samples                                                                   \
                                                 elapsed_time_for_sampling   
model n_diffusions n_solver_steps n_solver_order                             
advi  0.0          0.0            0.0                                 0.34   
ddpm  50.0         10.0           1.0                                 4.33   
                                  3.0                                 7.68   
                   20.0           1.0                                 6.36   
                                  3.0                                11.50   
      100.0        10.0           1.0                                 5.86   
                                  3.0                                 9.37   
                   20.0           1.0                                 9.77   
                                  3.0                                14.04   
flow  0.0          0.0            0.0                                 1.25   

n_samples                                               \
                                                   mse   
model n_diffusions n_solver_steps n_solver_order         
advi  0.0          0.0            0.0             8.59   
ddpm  50.0         10.0           1.0             0.86   
                                  3.0             0.89   
                   20.0           1.0             0.86   
                                  3.0             0.90   
      100.0        10.0           1.0             0.87   
                                  3.0             0.88   
                   20.0           1.0             0.89   
                                  3.0             0.88   
flow  0.0          0.0            0.0             0.89   

n_samples                                                             1000  \
                                                 elapsed_time_for_training   
model n_diffusions n_solver_steps n_solver_order                             
advi  0.0          0.0            0.0                                23.73   
ddpm  50.0         10.0           1.0                               307.72   
                                  3.0                               287.21   
                   20.0           1.0                               558.98   
                                  3.0                               465.90   
      100.0        10.0           1.0                               358.60   
                                  3.0                               310.56   
                   20.0           1.0                               641.08   
                                  3.0                               570.25   
flow  0.0          0.0            0.0                               472.70   

n_samples                                                                   \
                                                 elapsed_time_for_sampling   
model n_diffusions n_solver_steps n_solver_o

## Hierarchical model 1

\begin{equation}
\begin{split}
\gamma_i        & \sim  \text{Normal}(0, 1) \;\; \quad \forall i = 1, 2 \\
\beta_{ij}      & \sim  \text{Normal}(\gamma_i, 1)  \qquad \forall j = 1, \dots, 5  \\
\sigma    & \sim  \text{HalfNormal}(1) \\
y_{ijn}     & \sim \text{Normal}(\beta_{ij}, \sigma^2) \,\; \quad \forall n = 1, \dots, N
\end{split}
\end{equation}

In [8]:
hier_model = dfs[dfs.data_set == "hierarchical_model_1"].drop(columns=["data_set", "n_dim"]).groupby(["n_samples", "model", "n_diffusions", "n_solver_steps", "n_solver_order"]).mean().round(2)
hier_model = hier_model.reset_index()
hier_model = hier_model.pivot_table(index=["model", "n_diffusions", "n_solver_steps", "n_solver_order"], columns=["n_samples"])
hier_model = hier_model.swaplevel(0, 1, axis=1).sort_index(axis=1)
hier_model = hier_model.reindex([
    (100, "elapsed_time_for_training"),
    (100, "elapsed_time_for_sampling"),
    (100, "mse"),
    (1000, "elapsed_time_for_training"),
    (1000, "elapsed_time_for_sampling"),
    (1000, "mse")
], axis=1)
hier_model

n_samples                                                             100   \
                                                 elapsed_time_for_training   
model n_diffusions n_solver_steps n_solver_order                             
advi  0.0          0.0            0.0                                 6.77   
ddpm  50.0         10.0           1.0                                52.72   
                                  3.0                                59.40   
                   20.0           1.0                                85.75   
                                  3.0                                88.20   
      100.0        10.0           1.0                                58.08   
                                  3.0                                64.83   
                   20.0           1.0                               105.10   
                                  3.0                               103.22   
flow  0.0          0.0            0.0                                80.78   

n_samples                                                                   \
                                                 elapsed_time_for_sampling   
model n_diffusions n_solver_steps n_solver_order                             
advi  0.0          0.0            0.0                                 0.33   
ddpm  50.0         10.0           1.0                                 4.47   
                                  3.0                                 7.76   
                   20.0           1.0                                 6.91   
                                  3.0                                11.72   
      100.0        10.0           1.0                                 5.91   
                                  3.0                                 9.45   
                   20.0           1.0                                10.37   
                                  3.0                                15.18   
flow  0.0          0.0            0.0                                 1.20   

n_samples                                               \
                                                   mse   
model n_diffusions n_solver_steps n_solver_order         
advi  0.0          0.0            0.0             4.80   
ddpm  50.0         10.0           1.0             0.19   
                                  3.0             0.19   
                   20.0           1.0             0.19   
                                  3.0             0.19   
      100.0        10.0           1.0             0.19   
                                  3.0             0.19   
                   20.0           1.0             0.19   
                                  3.0             0.19   
flow  0.0          0.0            0.0             0.23   

n_samples                                                             1000  \
                                                 elapsed_time_for_training   
model n_diffusions n_solver_steps n_solver_order                             
advi  0.0          0.0            0.0                                23.52   
ddpm  50.0         10.0           1.0                               319.24   
                                  3.0                               281.85   
                   20.0           1.0                               572.51   
                                  3.0                               487.05   
      100.0        10.0           1.0                               391.41   
                                  3.0                               370.02   
                   20.0           1.0                               753.14   
                                  3.0                               604.08   
flow  0.0          0.0            0.0                               459.85   

n_samples                                                                   \
                                                 elapsed_time_for_sampling   
model n_diffusions n_solver_steps n_solver_o

## Hierarchical model 2

\begin{equation}
\begin{split}
\mu_\gamma        & \sim  \text{Normal}(0, 1) \\
\gamma_i        & \sim  \text{Normal}(\mu_\gamma, 1) \;\; \quad \forall i = 1, 2 \\
\beta_{ij}      & \sim  \text{Normal}(\gamma_i, 1)  \qquad \forall j = 1, \dots, 5  \\
\sigma    & \sim  \text{HalfNormal}(1) \\
y_{ijn}     & \sim \text{Normal}(\beta_{ij}, \sigma^2) \,\; \quad \forall n = 1, \dots, N
\end{split}
\end{equation}

In [9]:
hier_model = dfs[dfs.data_set == "hierarchical_model_2"].drop(columns=["data_set", "n_dim"]).groupby(["n_samples", "model", "n_diffusions", "n_solver_steps", "n_solver_order"]).mean().round(2)
hier_model = hier_model.reset_index()
hier_model = hier_model.pivot_table(index=["model", "n_diffusions", "n_solver_steps", "n_solver_order"], columns=["n_samples"])
hier_model = hier_model.swaplevel(0, 1, axis=1).sort_index(axis=1)
hier_model = hier_model.reindex([
    (100, "elapsed_time_for_training"),
    (100, "elapsed_time_for_sampling"),
    (100, "mse"),
    (1000, "elapsed_time_for_training"),
    (1000, "elapsed_time_for_sampling"),
    (1000, "mse")
], axis=1)
hier_model

n_samples                                                             100   \
                                                 elapsed_time_for_training   
model n_diffusions n_solver_steps n_solver_order                             
advi  0.0          0.0            0.0                                 7.29   
ddpm  50.0         10.0           1.0                                55.20   
                                  3.0                                59.26   
                   20.0           1.0                                86.11   
                                  3.0                                94.60   
      100.0        10.0           1.0                                61.05   
                                  3.0                                70.14   
                   20.0           1.0                               109.24   
                                  3.0                               101.82   
flow  0.0          0.0            0.0                                81.31   

n_samples                                                                   \
                                                 elapsed_time_for_sampling   
model n_diffusions n_solver_steps n_solver_order                             
advi  0.0          0.0            0.0                                 0.48   
ddpm  50.0         10.0           1.0                                 4.57   
                                  3.0                                 7.91   
                   20.0           1.0                                 7.04   
                                  3.0                                12.03   
      100.0        10.0           1.0                                 6.10   
                                  3.0                                10.03   
                   20.0           1.0                                10.68   
                                  3.0                                14.96   
flow  0.0          0.0            0.0                                 1.20   

n_samples                                               \
                                                   mse   
model n_diffusions n_solver_steps n_solver_order         
advi  0.0          0.0            0.0             5.91   
ddpm  50.0         10.0           1.0             0.56   
                                  3.0             0.56   
                   20.0           1.0             0.56   
                                  3.0             0.56   
      100.0        10.0           1.0             0.56   
                                  3.0             0.57   
                   20.0           1.0             0.56   
                                  3.0             0.57   
flow  0.0          0.0            0.0             0.59   

n_samples                                                             1000  \
                                                 elapsed_time_for_training   
model n_diffusions n_solver_steps n_solver_order                             
advi  0.0          0.0            0.0                                23.25   
ddpm  50.0         10.0           1.0                               329.90   
                                  3.0                               302.78   
                   20.0           1.0                               603.93   
                                  3.0                               490.44   
      100.0        10.0           1.0                               394.79   
                                  3.0                               354.46   
                   20.0           1.0                               766.47   
                                  3.0                               625.96   
flow  0.0          0.0            0.0                               552.86   

n_samples                                                                   \
                                                 elapsed_time_for_sampling   
model n_diffusions n_solver_steps n_solver_o

## Hierarchical model 3

\begin{equation}
\begin{split}
\sigma_\gamma   & \sim  \text{HalfNormal}(1) \\
\mu_\gamma      & \sim  \text{Normal}(0, 1) \\
\gamma_i        & \sim  \text{Normal}(\mu_\gamma, \sigma^2_\gamma) \;\; \quad \forall i = 1, 2 \\
\beta_{ij}      & \sim  \text{Normal}(\gamma_i, 1)  \qquad \forall j = 1, \dots, 5  \\
\sigma    & \sim  \text{HalfNormal}(1) \\
y_{ijn}     & \sim \text{Normal}(\beta_{ij}, \sigma^2) \,\; \quad \forall n = 1, \dots, N
\end{split}
\end{equation}

In [10]:
hier_model = dfs[dfs.data_set == "hierarchical_model_3"].drop(columns=["data_set", "n_dim"]).groupby(["n_samples", "model", "n_diffusions", "n_solver_steps", "n_solver_order"]).mean().round(2)
hier_model = hier_model.reset_index()
hier_model = hier_model.pivot_table(index=["model", "n_diffusions", "n_solver_steps", "n_solver_order"], columns=["n_samples"])
hier_model = hier_model.swaplevel(0, 1, axis=1).sort_index(axis=1)
hier_model = hier_model.reindex([
    (100, "elapsed_time_for_training"),
    (100, "elapsed_time_for_sampling"),
    (100, "mse"),
    (1000, "elapsed_time_for_training"),
    (1000, "elapsed_time_for_sampling"),
    (1000, "mse")
], axis=1)
hier_model

n_samples                                                             100   \
                                                 elapsed_time_for_training   
model n_diffusions n_solver_steps n_solver_order                             
advi  0.0          0.0            0.0                                 7.56   
ddpm  50.0         10.0           1.0                                59.20   
                                  3.0                                60.04   
                   20.0           1.0                                86.33   
                                  3.0                                95.32   
      100.0        10.0           1.0                                63.37   
                                  3.0                                67.96   
                   20.0           1.0                               103.01   
                                  3.0                               105.09   
flow  0.0          0.0            0.0                                90.72   

n_samples                                                                   \
                                                 elapsed_time_for_sampling   
model n_diffusions n_solver_steps n_solver_order                             
advi  0.0          0.0            0.0                                 0.35   
ddpm  50.0         10.0           1.0                                 4.73   
                                  3.0                                 7.95   
                   20.0           1.0                                 7.13   
                                  3.0                                12.07   
      100.0        10.0           1.0                                 5.99   
                                  3.0                                 9.71   
                   20.0           1.0                                10.12   
                                  3.0                                15.20   
flow  0.0          0.0            0.0                                 1.32   

n_samples                                               \
                                                   mse   
model n_diffusions n_solver_steps n_solver_order         
advi  0.0          0.0            0.0             5.94   
ddpm  50.0         10.0           1.0             0.77   
                                  3.0             0.69   
                   20.0           1.0             0.82   
                                  3.0             0.70   
      100.0        10.0           1.0             0.71   
                                  3.0             0.59   
                   20.0           1.0             0.67   
                                  3.0             0.63   
flow  0.0          0.0            0.0             0.39   

n_samples                                                             1000  \
                                                 elapsed_time_for_training   
model n_diffusions n_solver_steps n_solver_order                             
advi  0.0          0.0            0.0                                42.50   
ddpm  50.0         10.0           1.0                               366.97   
                                  3.0                               298.32   
                   20.0           1.0                               594.04   
                                  3.0                               519.81   
      100.0        10.0           1.0                               418.98   
                                  3.0                               388.37   
                   20.0           1.0                               779.95   
                                  3.0                               664.93   
flow  0.0          0.0            0.0                               484.22   

n_samples                                                                   \
                                                 elapsed_time_for_sampling   
model n_diffusions n_solver_steps n_solver_o

## Hierarchical model 4

\begin{equation}
\begin{split}
\sigma_\gamma   & \sim  \text{HalfNormal}(1) \\
\mu_\gamma      & \sim  \text{Normal}(0, 1) \\
\gamma_i        & \sim  \text{Normal}(\mu_\gamma, \sigma^2_\gamma) \;\; \quad \forall i = 1, 2 \\
\sigma_\beta    & \sim  \text{HalfNormal}(1) \\
\beta_{ij}      & \sim  \text{Normal}(\gamma_i, \sigma^2_\beta)  \qquad \forall j = 1, \dots, 5  \\
y_{ijn}     & \sim \text{Normal}(\beta_{ij}, 1) \,\; \quad \forall n = 1, \dots, N
\end{split}
\end{equation}

In [11]:
hier_model = dfs[dfs.data_set == "hierarchical_model_4"].drop(columns=["data_set", "n_dim"]).groupby(["n_samples", "model", "n_diffusions", "n_solver_steps", "n_solver_order"]).mean().round(2)
hier_model = hier_model.reset_index()
hier_model = hier_model.pivot_table(index=["model", "n_diffusions", "n_solver_steps", "n_solver_order"], columns=["n_samples"])
hier_model = hier_model.swaplevel(0, 1, axis=1).sort_index(axis=1)
hier_model = hier_model.reindex([
    (100, "elapsed_time_for_training"),
    (100, "elapsed_time_for_sampling"),
    (100, "mse"),
    (1000, "elapsed_time_for_training"),
    (1000, "elapsed_time_for_sampling"),
    (1000, "mse")
], axis=1)
hier_model

n_samples                                                             100   \
                                                 elapsed_time_for_training   
model n_diffusions n_solver_steps n_solver_order                             
advi  0.0          0.0            0.0                                 7.74   
ddpm  50.0         10.0           1.0                                50.73   
                                  3.0                                59.55   
                   20.0           1.0                                90.13   
                                  3.0                                89.73   
      100.0        10.0           1.0                                58.67   
                                  3.0                                67.05   
                   20.0           1.0                               102.74   
                                  3.0                               107.99   
flow  0.0          0.0            0.0                                87.41   

n_samples                                                                   \
                                                 elapsed_time_for_sampling   
model n_diffusions n_solver_steps n_solver_order                             
advi  0.0          0.0            0.0                                 0.36   
ddpm  50.0         10.0           1.0                                 4.42   
                                  3.0                                 7.75   
                   20.0           1.0                                 7.11   
                                  3.0                                11.73   
      100.0        10.0           1.0                                 5.96   
                                  3.0                                 9.67   
                   20.0           1.0                                10.51   
                                  3.0                                15.53   
flow  0.0          0.0            0.0                                 1.25   

n_samples                                                \
                                                    mse   
model n_diffusions n_solver_steps n_solver_order          
advi  0.0          0.0            0.0             12.47   
ddpm  50.0         10.0           1.0              0.63   
                                  3.0              0.63   
                   20.0           1.0              0.64   
                                  3.0              0.63   
      100.0        10.0           1.0              0.63   
                                  3.0              0.55   
                   20.0           1.0              0.63   
                                  3.0              0.58   
flow  0.0          0.0            0.0              0.64   

n_samples                                                             1000  \
                                                 elapsed_time_for_training   
model n_diffusions n_solver_steps n_solver_order                             
advi  0.0          0.0            0.0                                40.12   
ddpm  50.0         10.0           1.0                               353.61   
                                  3.0                               290.16   
                   20.0           1.0                               595.12   
                                  3.0                               501.43   
      100.0        10.0           1.0                               443.80   
                                  3.0                               337.37   
                   20.0           1.0                               713.09   
                                  3.0                               600.31   
flow  0.0          0.0            0.0                               555.45   

n_samples                                                                   \
                                                 elapsed_time_for_sampling   
model n_diffusions n_solver_ste

## Hierarchical model 5

\begin{equation}
\begin{split}
\sigma_\gamma   & \sim  \text{HalfNormal}(1) \\
\mu_\gamma      & \sim  \text{Normal}(0, 1) \\
\gamma_i        & \sim  \text{Normal}(\mu_\gamma, \sigma^2_\gamma) \;\; \quad \forall i = 1, \dots, 5 \\
\sigma_\beta    & \sim  \text{HalfNormal}(1) \\
\beta_{ij}      & \sim  \text{Normal}(\gamma_i, \sigma^2_\beta)  \qquad \forall j = 1, 2  \\
y_{ijn}     & \sim \text{Normal}(\beta_{ij}, 1) \,\; \quad \forall n = 1, \dots, N
\end{split}
\end{equation}

In [12]:
hier_model = dfs[dfs.data_set == "hierarchical_model_5"].drop(columns=["data_set", "n_dim"]).groupby(["n_samples", "model", "n_diffusions", "n_solver_steps", "n_solver_order"]).mean().round(2)
hier_model = hier_model.reset_index()
hier_model = hier_model.pivot_table(index=["model", "n_diffusions", "n_solver_steps", "n_solver_order"], columns=["n_samples"])
hier_model = hier_model.swaplevel(0, 1, axis=1).sort_index(axis=1)
hier_model = hier_model.reindex([
    (100, "elapsed_time_for_training"),
    (100, "elapsed_time_for_sampling"),
    (100, "mse"),
    (1000, "elapsed_time_for_training"),
    (1000, "elapsed_time_for_sampling"),
    (1000, "mse")
], axis=1)
hier_model

n_samples                                                             100   \
                                                 elapsed_time_for_training   
model n_diffusions n_solver_steps n_solver_order                             
advi  0.0          0.0            0.0                                 7.38   
ddpm  50.0         10.0           1.0                                57.18   
                                  3.0                                54.52   
                   20.0           1.0                                92.67   
                                  3.0                                83.29   
      100.0        10.0           1.0                                66.07   
                                  3.0                                64.50   
                   20.0           1.0                               110.28   
                                  3.0                               107.82   
flow  0.0          0.0            0.0                                96.01   

n_samples                                                                   \
                                                 elapsed_time_for_sampling   
model n_diffusions n_solver_steps n_solver_order                             
advi  0.0          0.0            0.0                                 0.37   
ddpm  50.0         10.0           1.0                                 4.69   
                                  3.0                                 7.17   
                   20.0           1.0                                 7.70   
                                  3.0                                10.83   
      100.0        10.0           1.0                                 6.29   
                                  3.0                                 9.22   
                   20.0           1.0                                10.55   
                                  3.0                                15.53   
flow  0.0          0.0            0.0                                 1.17   

n_samples                                                \
                                                    mse   
model n_diffusions n_solver_steps n_solver_order          
advi  0.0          0.0            0.0             11.77   
ddpm  50.0         10.0           1.0              0.49   
                                  3.0              0.46   
                   20.0           1.0              0.51   
                                  3.0              0.52   
      100.0        10.0           1.0              0.41   
                                  3.0              0.39   
                   20.0           1.0              0.40   
                                  3.0              0.44   
flow  0.0          0.0            0.0              0.62   

n_samples                                                             1000  \
                                                 elapsed_time_for_training   
model n_diffusions n_solver_steps n_solver_order                             
advi  0.0          0.0            0.0                                38.87   
ddpm  50.0         10.0           1.0                               276.22   
                                  3.0                               266.37   
                   20.0           1.0                               546.03   
                                  3.0                               491.99   
      100.0        10.0           1.0                               437.52   
                                  3.0                               353.74   
                   20.0           1.0                               757.28   
                                  3.0                               605.38   
flow  0.0          0.0            0.0                               689.78   

n_samples                                                                   \
                                                 elapsed_time_for_sampling   
model n_diffusions n_solver_ste